# Demand flexibility optimization
Simple scenario to minimize variance of demand in general, with no co-optimization

In [1]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")

    Updating registry at `C:\Users\zoele\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zoele\.julia\environments\v1.8\Manifest.toml`


In [2]:
using CSV, DataFrames, JuMP, Gurobi, Plots

## Imports

In [3]:
demand = CSV.read("demand.csv", DataFrame);
print(size(demand))
first(demand, 5)

(8760, 4)

Row,Commercial,Industrial,Residential,Transportation
,Float64,Float64,Float64,Float64
1,1.7772e5,1.09564e5,1.69055e5,55394.9
2,1.74776e5,108330.0,1.64425e5,37636.1
3,1.73069e5,1.02035e5,1.6518e5,33108.4
4,1.71107e5,96909.0,1.68945e5,29324.8
5,1.71561e5,94888.4,1.7368e5,23657.3


In [4]:
flexibility = CSV.read("flex.csv", DataFrame)
first(flexibility, 5)

Row,Commercial,Industrial,Residential,Transportation
,Float64,Float64,Float64,Float64
1,2349.96,5093.49,15203.1,31358.9
2,2332.55,5023.94,9442.63,19418.7
3,2349.94,4774.39,5711.2,16340.4
4,2369.11,4578.63,5115.78,13844.4
5,2444.76,4501.5,4419.4,10062.8


In [5]:
flexibility_settings = Matrix(CSV.read("flex_settings.csv", DataFrame))

4×1 Matrix{Int64}:
 7
 5
 8
 7

## Model setup

In [6]:
hrs = 8760
n_eu = 4

D_b = demand[1:hrs, 1:n_eu] # Baseline demand with no flex
print(size(D_b))

A_f = flexibility[1:hrs, 1:n_eu]
H_shift = flexibility_settings;

# pad year by 1 day
add_h = 24
D_b = vcat(demand[8761-add_h:8760, 1:n_eu], D_b, D_b[1:24, 1:n_eu])
A_f = vcat(flexibility[8761-add_h:8760, 1:n_eu], A_f, A_f[1:24, 1:n_eu])
print(size(D_b))
total_demand = sum(eachcol(D_b))
print(size(total_demand))

(8760, 4)(8808, 4)(8808,)

In [7]:
model = Model(Gurobi.Optimizer)
# set_optimizer_attribute(model, "NonConvex", 2)
# set_optimizer_attribute(model, "IterationLimit", 6000)
println("Model: initialized!\n")

T = size(D_b, 1) # hours in simulation
println("\nTimespan: ", T)
time_ls = add_h+1:hrs+add_h
println(time_ls)
println()

# Hourly Demand - total
@variable(model, X_D[1:T] >= 0);

# Demand with flexibility 
@variable(model, X_F[1:T, 1:4] <= 0); # flexibility used
@variable(model, X_S[1:T, 1:4] >= 0); # shift
# @variable(model, H_shift[1:T, 1:4] >= 0); # To start assume shift amounts are prescriptive and forward

println("Demand constraint")
@constraint(model, Con_d, sum(sum(eachcol(X_D))) == sum(total_demand));

println("Flexibility constraint")
# demand side flexibility cannot exeed available for each sector at each hour
@constraint(model, [i = 1:4, t = 1:T], X_F[t, i] >= -A_f[t, i])

println("Demand-side flexibility and demand") 
# total demand at each hour must match realized demand summed across sectors minus flexibility summed across sectors
@constraint(model, [t = time_ls], X_D[t] == total_demand[t] + sum(eachcol(X_F))[t] + sum(eachcol(X_S))[t]);
# @constraint(model, [t = time_ls], total_demand[t] == X_D[t] + sum(eachcol(X_F))[t]);

println("Demand-side flexibility shifting") 
# Shift is constrained by hours shifted
@constraint(model, [i = 1:4, t = time_ls], X_S[t, i] == -X_F[t-H_shift[i], i]);

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2023-12-11
Model: initialized!


Timespan: 8808
25:8784

Demand constraint
Flexibility constraint
Demand-side flexibility and demand
Demand-side flexibility shifting


#### Objective

In [8]:
# Minimize variance of demand (least squares)
mean = sum(total_demand)/length(total_demand)
println("Mean daily demand: ", mean)
@objective(model, Min, 
    (
        sum((X_D[t] - mean)^2 for t in 1:T)
    ));

Mean daily demand: 661825.4944104723


In [9]:
#Optimize
optimize!(model)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 79033 rows, 79272 columns and 192960 nonzeros
Model fingerprint: 0x7ed62f85
Model has 8808 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+06, 1e+06]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 6e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 70272 rows and 44188 columns
Presolve time: 0.11s
Presolved: 8761 rows, 35084 columns, 70104 nonzeros
Presolved model has 8808 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.502e+04
 Factor NZ  : 1.662e+05 (roughly 20 MB of memory)
 Factor Ops : 3.286e+06 (less than 1 second per iter

In [10]:
plotly()
plot(time_ls, total_demand[time_ls], label="Demand", lc=:black, lw=1, ls=:dot)
plot!(time_ls, value.(X_D)[time_ls], label="Demand, flexible", lc=:black, lw=2)
plot!(time_ls, sum(eachcol(value.(X_F)))[time_ls], label="Flexibility, used", lc=:green, lw=1)
plot!(time_ls, sum(eachcol(value.(X_S)))[time_ls], label="Flexibility, shifted", lc=:red, lw=1)
plot!(size=(1400,400))

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err = ArgumentError("Package PlotlyBase not found in current path.\n- Run `import Pkg; Pkg.add(\"PlotlyBase\")` to install the PlotlyBase package.")
└ @ Plots C:\Users\zoele\.julia\packages\Plots\M4dfL\src\backends.jl:545


In [11]:
CSV.write("demand_flex.csv", Tables.table(value.(X_D)[time_ls]), writeheader=false)

"demand_flex.csv"

: 